# 1. Installing dependencies and imports

In [ ]:
pip install 123-object-detection

In [ ]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

## 1.1. Need to clone repo when using colab

In [ ]:
!git clone https://github.com/Slimmerd/car-detection.git

# 2. Defining Paths and links

In [ ]:
# TODO Paste your pre-trained model url
PRET_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

MAIN_PATH = os.getcwd() + '/car-detection'
MODEL_PATH = MAIN_PATH +'/models'
UTILS_PATH = MAIN_PATH + '/utils'

DATASET_PATH = MAIN_PATH + '/dataset'
DATASET_NAME = 'object-detection-crowdai'

PRET_MODEL_PATH = MAIN_PATH + '/models/pretrained'
CUST_MODEL_PATH = MAIN_PATH + '/models/custom'

# TODO Paste your pre-trained model name
PRET_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
# TODO Name your custom model
CUST_MODEL_NAME = '8k_model'

CFG_PATH = CUST_MODEL_PATH + '/' + CUST_MODEL_NAME + '/pipeline.config'

# 3. Download dataset and pre-trained model

## 3.1. Creating folders and downloading, unpacking dataset

In [ ]:
!mkdir {DATASET_PATH}
!cd {DATASET_PATH} && wget https://s3.amazonaws.com/udacity-sdc/annotations/object-detection-crowdai.tar.gz
!cd {DATASET_PATH} && tar xf {DATASET_NAME +'.tar.gz'}

## 3.2. Creating folders and downloading, unpacking pre-trained model

In [ ]:
!mkdir {MODEL_PATH}
!mkdir {PRET_MODEL_PATH}

!cd {PRET_MODEL_PATH} && wget {PRET_MODEL_URL}
!cd {PRET_MODEL_PATH} && tar xf {PRET_MODEL_NAME+'.tar.gz'}

## 3.3. Creating folders and copying pipeline config

In [ ]:
!mkdir {CUST_MODEL_PATH}
!mkdir {CUST_MODEL_PATH + '/' + CUST_MODEL_NAME}
!cp {PRET_MODEL_PATH +'/' + PRET_MODEL_NAME + '/pipeline.config'} {CUST_MODEL_PATH + '/' + CUST_MODEL_NAME}

# 4. Divide dataset to train and test datasets

In [ ]:
ds = pd.read_csv(UTILS_PATH + '/labels_crowdai.csv')
ds.head()

# 4.1 Select input size

In [ ]:
gb = ds.groupby('Frame')
gb_list = [gb.get_group(x) for x in gb.groups]

# TODO Amount of unique frames in dataset
#(max size 9218, test and train should be 20% different train < test)
train_index = np.random.choice(len(gb_list), size=7374, replace=False)
test_index = np.setdiff1d(list(range(9218)), train_index)

In [ ]:
len(train_index), len(test_index)

In [ ]:
train = pd.concat([gb_list[i] for i in train_index])
test = pd.concat([gb_list[i] for i in test_index])
len(train), len(test)

# Creating separate datasets
train.to_csv(DATASET_PATH + '/train_labels.csv', index=None)
test.to_csv(DATASET_PATH + '/test_labels.csv', index=None)

# 5. Creating Label map

In [ ]:
labels = [{'name':'Car', 'id':1}, {'name':'Truck', 'id':2}, {'name':'Pedestrian', 'id':3}]

with open(DATASET_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 6. Creating TFRecords

In [ ]:
!python {UTILS_PATH + '/TFRecord.py'} -image_dir {DATASET_PATH + '/' + DATASET_NAME} -csv_input {DATASET_PATH + '/train_labels.csv'} -output_path {DATASET_PATH + '/train.record'}
!python {UTILS_PATH + '/TFRecord.py'} -image_dir {DATASET_PATH + '/' + DATASET_NAME} -csv_input {DATASET_PATH + '/test_labels.csv'} -output_path {DATASET_PATH + '/test.record'}

# 7. Configuring config

In [ ]:
cfg = config_util.get_configs_from_pipeline_file(CFG_PATH)

In [ ]:
cfg

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CFG_PATH, "r") as f:
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRET_MODEL_PATH +'/' + PRET_MODEL_NAME +'/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= DATASET_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [DATASET_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = DATASET_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [DATASET_PATH + '/train.record']

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CFG_PATH, "wb") as f:
    f.write(config_text)  

# 8. Train model

In [ ]:
TRAINER = UTILS_PATH + '/model_main_tf2.py'
# Recommended amount of steps is 5000, however it depends on your hardware performance
# Minimum amount get a normal results is 2000
# Optimal is 8000
train_command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINER, CUST_MODEL_PATH + '/' + CUST_MODEL_NAME,CFG_PATH)

In [ ]:
print(train_command)

In [ ]:
!{train_command}

# 9. Export model

In [ ]:
!tar -czf models.tar.gz {CUST_MODEL_PATH + '/' + CUST_MODEL_NAME}

In [ ]:
# Open files folder in colab and transfer your model to drive/MyDrive if you want to save it
from google.colab import drive
drive.mount('/content/drive')

# 10. Import model

In [ ]:
# Don't forget change CUST_MODEL_NAME to the model you importing (if you didn't)

In [ ]:
!tar -zxvf {'models'+'.tar.gz'}

# 11. Load Model

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CFG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)

# Check how many checkpoints in your model it might be ckpt-2, ckpt-5 etc
# TODO Select highest model checkpoint
ckpt.restore(CUST_MODEL_PATH + '/' + CUST_MODEL_NAME + '/ckpt-3').expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 12. Detect from picture

In [ ]:
import cv2
import numpy as np
from PIL import Image

## 12.1. Select picture

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(DATASET_PATH + '/label_map.pbtxt')

# TODO You can add your image if so you should change path to your image
# IMAGE_PATH = DATASET_PATH + '/' + DATASET_NAME +'/1479498372942264998.jpg'
# IMAGE_PATH = DATASET_PATH + '/people.jpeg'
# IMAGE_PATH = DATASET_PATH + '/test2.jpg'
IMAGE_PATH = DATASET_PATH + '/lorry.jpg'

## 12.2. Generate picture

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

# TODO max_boxes_to_draw is the amount of things it can detect on picture
# TODO min_score_thresh=0.7 (0-1) shows on the picture if recognised percent is 70%
viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.5,
            agnostic_mode=False)

Image.fromarray(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))

# 13. Loading stats

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=car-detection/models/custom/custom_based_tpu8

In [ ]:
!tensorboard dev upload \
  --logdir car-detection/models/custom/custom_based_tpu8 \
  --name "(optional) My latest experiment" \
  --description "(optional) Simple comparison of several hyperparameters" \
  --one_shot